In [14]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
sys.path.append('../models')
sys.path.append('../fit')

from model import LSTMSentiment
from torchtext import data
from torchtext import datasets
import torch
import cd

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load dset + model

In [ ]:
# data params
vector_cache =  '../data/.vector_cache/input_vectors.pt'
word_vectors ='glove.6B.300d'
batch_size = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'


# load dset
inputs = data.Field(lower= True)
answers = data.Field(sequential=False, unk_token=None)
train, dev, test = datasets.SST.splits(inputs, answers, fine_grained=False, train_subtrees=True,
                                       filter_pred=lambda ex: ex.label != 'neutral')
inputs.build_vocab(train, dev, test)

if os.path.isfile(vector_cache):
    inputs.vocab.vectors = torch.load(vector_cache)
else:
    inputs.vocab.load_vectors(word_vectors)
    os.makedirs(os.path.dirname(vector_cache), exist_ok=True)
    torch.save(inputs.vocab.vectors,vector_cache)
answers.build_vocab(train)

train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_size=batch_size, device=torch.device(0))

# load model
model_path = "../models/init_models"
model_list = os.listdir(model_path)
model1 = torch.load(os.path.join(model_path, model_list[0]), map_location=torch.device(0)).eval()

model2 = torch.load(os.path.join(model_path, model_list[1]), map_location=torch.device(0)).eval()

# evaluate cd word-level

In [ ]:
# choose hyperparams
it = dev_iter
m = model1

it.init_epoch()
# check out how two models differ
import torch.optim as O
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
n_dev_correct, dev_loss = 0, 0
for dev_batch_idx, dev_batch in enumerate(it):
    answer1 = m(dev_batch)
    batch_length = dev_batch.text.shape[0]
    batch_num = dev_batch.text.shape[1]

    start = np.random.randint(batch_length-1)
    stop = start + np.random.randint(batch_length-start)
    rel, _ =cd.cd_batch_text(dev_batch, m, start, stop)
    print(rel)
#     n_dev_correct += (((torch.max(answer2, 1)[1].view(dev_batch.label.size()).data == dev_batch.label.data))).sum()
#     dev_loss = criterion(answer2, dev_batch.label)
# dev_acc = 100. * n_dev_correct / len(dev)
# print(dev_acc.item(), dev_loss.item())